In [1]:
using Revise

In [276]:
using RigidBodyDynamics
using Rotations
using AtlasRobot
using LearningMPC.Models
using StaticArrays
using MeshCat
using MeshCatMechanisms
using Blink
using CoordinateTransformations

In [213]:
vis = Visualizer()
open(vis, Window());

Blink.AtomShell.Window(2, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 2040`, ProcessRunning), TCPSocket(RawFD(59) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(2, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(67) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 2, Nullable{Any}(true))))

In [591]:
collect(Iterators.flatten(position_bounds(j) for j in joints(boxatlas)))

11-element Array{RigidBodyDynamics.Bounds{Float64},1}:
 (-10.0, 10.0)    
 (-10.0, 10.0)    
 (-10.0, 10.0)    
 (0.0, 1.6708)    
 (0.0, 1.6708)    
 (-0.05, 0.785398)
 (-0.05, 0.785398)
 (0.4, 0.8)       
 (0.4, 0.8)       
 (0.475, 0.95)    
 (0.475, 0.95)    

In [303]:
atlas = AtlasRobot.mechanism()

modifications = [
    ("r_hand_mount", "r_hand", RotZYX(π, -π/2, 0), SVector(0, -0.195, 0)),
    ("l_hand_mount", "l_hand", RotZYX(π, -π/2, 0), SVector(0, -0.195, 0)),
    ("r_foot_sole", "r_foot", RotZYX(0., 0., 0.), SVector(0.0426, -0.0017, -0.07645)),
    ("l_foot_sole", "l_foot", RotZYX(0., 0., 0.), SVector(0.0426, 0.0017, -0.07645)),   
]

for (bodyname, basename, rot, trans) in modifications
    base = findbody(atlas, basename)
    frame = CartesianFrame3D(bodyname)
    inertia = SpatialInertia(frame, SDiagonal(0., 0, 0), SVector(0., 0, 0), 0.0)
    body = RigidBody(inertia)
    joint = Joint("$(basename)_to_$(bodyname)", Fixed{Float64}())
    before_joint = Transform3D(frame_before(joint), default_frame(base), rot, trans)
    attach!(atlas, base, body, joint, joint_pose=before_joint)
end

function init_atlas!(state::MechanismState)
    mechanism = state.mechanism
    zero!(state)
    kneebend = 1.1
    hipbendextra = 0.1
    for sideprefix in ('l', 'r')
        knee = findjoint(mechanism, "$(sideprefix)_leg_kny")
        hippitch = findjoint(mechanism, "$(sideprefix)_leg_hpy")
        anklepitch = findjoint(mechanism, "$(sideprefix)_leg_aky")        
        set_configuration!(state, knee, [kneebend])
        set_configuration!(state, hippitch, [-kneebend / 2 + hipbendextra])
        set_configuration!(state, anklepitch, [-kneebend / 2 - hipbendextra])
    end
    floatingjoint = first(out_joints(root_body(mechanism), mechanism))
    set_configuration!(state, floatingjoint, [1; 0; 0; 0; 0; 0; 0.85])
    state
end


init_atlas! (generic function with 1 method)

In [322]:
boxatlas = mechanism(BoxAtlas());

In [323]:
delete!(vis)
mvis1 = MechanismVisualizer(atlas, Skeleton(randomize_colors=true), vis[:atlas])
mvis2 = MechanismVisualizer(boxatlas, Skeleton(randomize_colors=true), vis[:boxatlas])
set_configuration!(mvis2, configuration(nominal_state(BoxAtlas())))
set_configuration!(mvis1, configuration(init_atlas!(MechanismState(atlas))))
settransform!(vis[:atlas], Translation(-1, 0, 0.))

matching_bodies =  ["pelvis", "r_foot_sole", "l_foot_sole", "r_hand_mount", "l_hand_mount"]

for body in matching_bodies
    for mvis in [mvis1, mvis2]
        setelement!(mvis, default_frame(findbody(mvis.state.mechanism, body)))
    end
end

In [367]:
function bodymap(state, bodynames)
    mechanism = state.mechanism
    nq = num_positions(state)
    nv = num_velocities(state)
    A = zeros(12 * length(bodynames), nq + nv)
    b = zeros(12 * length(bodynames))
    J_q_to_v = RigidBodyDynamics.configuration_derivative_to_velocity_jacobian(state)
    for i in eachindex(bodynames)
        body = findbody(mechanism, bodynames[i])
        path_to_body = path(mechanism, root_body(mechanism), body)
        J_geometric = geometric_jacobian(state, path_to_body)
        J_analytic = Array(J_geometric) * Array(J_q_to_v)
        rows = (12 * (i - 1)) + (1:12)
        A[rows[1:6], 1:nq] .= J_analytic
        A[rows[7:9], nq + (1:nv)] .= angular(J_geometric)
        A[rows[10:12], nq + (1:nv)] .= linear(J_geometric)
        H = transform_to_root(state, body)
        r = RodriguesVec(rotation(H))
        b[rows[1:3]] .= SVector(r.sx, r.sy, r.sz)
        b[rows[4:6]] .= translation(H)
        T = twist_wrt_world(state, body)
        b[rows[7:9]] .= angular(T)
        b[rows[10:12]] .= linear(T)
    end
    b .= b .- A * Vector(state)
    A, b
end

function _unwrap(b1::Real, b2::Real)
    b1 + (mod(b2 - b1 + π, 2π) - π)
end
    

function _unwrap!(b2::AbstractVector, b1::AbstractVector)
    for i in 1:(length(b2) ÷ 12)
        rows = (12 * (i - 1)) + (1:12)
        for row in vcat(rows[1:3], rows[7:9])
            b2[row] = _unwrap(b1[row], b2[row])
            @assert(b1[row] - π <= b2[row] <= b1[row] + π, (b1[row], b2[row]))
        end
    end
end

_unwrap! (generic function with 1 method)

In [540]:
reload("IKMimic")

mutable struct IKMimicWorkspace{T} # /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IKMimic/src/IKMimic.jl, line 119:
    reference::BodyMap{T} # /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IKMimic/src/IKMimic.jl, line 120:
    result::BodyMap{T} # /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IKMimic/src/IKMimic.jl, line 121:
    H::Matrix{T} # /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IKMimic/src/IKMimic.jl, line 122:
    H_fact::LinAlg.Cholesky{T, Matrix{T}} # /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IKMimic/src/IKMimic.jl, line 123:
    p1::Vector{T} # /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IKMimic/src/IKMimic.jl, line 124:
    f1::Vector{T} # /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IKMimic/src/IKMimic.jl, line 125:
    f2::Vector{T} # /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IKMimic/src/IKMimic.j

In [565]:
state1 = init_atlas!(MechanismState(atlas))
state2 = MechanismState(boxatlas)
matching_bodies =  ["pelvis", "r_foot_sole", "l_foot_sole", "r_hand_mount", "l_hand_mount"]

m1 = IKMimic.BodyMap(state1, matching_bodies)
m2 = IKMimic.BodyMap(state2, matching_bodies);
work = IKMimic.IKMimicWorkspace(m1, m2);

In [555]:
for i in 1:20
    IKMimic.ik_mimic!(state2, state1, work)
    set_configuration!(mvis2, configuration(state2))
    sleep(0.05)
end
    

In [561]:
IKMimic.sensitivity(work)[1:10, 1:7]

10×7 Array{Float64,2}:
 0.0   0.042099      9.98344e-17   3.20779e-19  …   1.0           9.53947e-18
 0.0   0.0014739    -4.65887e-18  -1.34245e-20      2.92748e-17   1.0        
 0.0   2.0          -6.44603e-16  -1.83423e-18      5.91285e-17  -1.86067e-17
 0.0  -4.19804e-15  -7.06371e-16  -2.41053e-18      2.71362e-16  -1.6015e-16 
 0.0   3.94877e-15   7.13113e-16   2.43454e-18      4.03505e-19  -9.4167e-17 
 0.0  -2.96878e-15  -5.95856e-16  -2.16413e-18  …   1.19343e-17  -1.88322e-17
 0.0   2.70893e-15   5.95355e-16   2.16195e-18     -1.27512e-17   7.35113e-18
 0.0   3.88925e-15   5.67748e-16   1.86519e-18     -3.48423e-16   2.03923e-16
 0.0  -3.69852e-15  -5.82968e-16  -1.91854e-18     -4.08886e-18   1.26393e-16
 0.0   2.7247e-16    6.03371e-17   2.25533e-19      2.83609e-17  -2.32749e-16

In [530]:
using BenchmarkTools
@btime IKMimic.ik_mimic!($state2, $state1, $work)

  41.234 μs (6 allocations: 144 bytes)


In [463]:
@btime IKMimic.mimic_update!($m2, $m1)

  25.599 μs (13 allocations: 10.34 KiB)


MechanismState{Float64, Float64, Float64, …}(…)

In [575]:
vis3 = Visualizer()
open(vis3, Window())

mvis3 = MechanismVisualizer(boxatlas, Skeleton(randomize_colors=true), vis3[:atlas3])
mvis4 = MechanismVisualizer(boxatlas, Skeleton(randomize_colors=true), vis3[:atlas4])
settransform!(vis3[:atlas4], Translation(-1, 0, 0))

MeshCat Visualizer with path /meshcat/atlas4

In [588]:
state3 = MechanismState(boxatlas)
state4 = MechanismState(boxatlas)
rand!(state3)
rand!(state4)
work3 = IKMimic.IKMimicWorkspace(state3, state4, ["pelvis",
        "r_foot_sole",
        "l_foot_sole",
        "r_hand_mount",
        "l_hand_mount"])
set_configuration!(mvis3, configuration(state3))
set_configuration!(mvis4, configuration(state4))

In [589]:
for i in 1:20
    IKMimic.ik_mimic!(state4, state3, work3)
    set_configuration!(mvis4, configuration(state4))
    sleep(0.05)
end
    